# 机器学习工程师纳米学位
## 监督学习
## 项目 2: 搭建一个学生干预系统

欢迎来到机器学习工程师纳米学位的第二个项目！在此文件中，有些示例代码已经提供给你，但你还需要实现更多的功能让项目成功运行。除非有明确要求，你无须修改任何已给出的代码。以**'练习'**开始的标题表示接下来的代码部分中有你必须要实现的功能。每一部分都会有详细的指导，需要实现的部分也会在注释中以**'TODO'**标出。请仔细阅读所有的提示！

除了实现代码外，你还**必须**回答一些与项目和你的实现有关的问题。每一个需要你回答的问题都会以**'问题 X'**为标题。请仔细阅读每个问题，并且在问题后的**'回答'**文字框中写出完整的答案。我们将根据你对问题的回答和撰写代码所实现的功能来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 问题 1 - 分类 vs. 回归
*在这个项目中你的任务是找出那些如果不给予帮助，最重可能无法毕业的学生。你觉得这个问题是哪种类型的监督学习问题，是分类问题还是回归问题？为什么？*

**答案: **
是分类问题。针对项目中的任务构建的模型的预测输出是二元的：可以毕业 & 无法毕业。

## 分析数据
运行下面区域的代码以载入学生数据集，以及一些此项目所需的Python库。注意数据集的最后一列`'passed'`是我们的预测的目标（表示学生是毕业了还是没有毕业），其他的列是每个学生的属性。

In [1]:
# 载入所需要的库
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# 载入学生数据集
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
#print student_data

Student data read successfully!


### 练习: 分析数据
我们首先通过调查数据，以确定有多少学生的信息，并了解这些学生的毕业率。在下面的代码单元中，你需要完成如下的运算：
- 学生的总数， `n_students`。
- 每个学生的特征总数， `n_features`。
- 毕业的学生的数量， `n_passed`。
- 未毕业的学生的数量， `n_failed`。
- 班级的毕业率， `grad_rate`， 用百分数表示(%)。


In [2]:
print student_data.shape
# TODO： 计算学生的数量
n_students = student_data.shape[0]

# TODO： 计算特征数量
n_features = student_data.shape[1] - 1

# TODO： 计算通过的学生数
n_passed = student_data[student_data['passed']=='yes'].shape[0]

# TODO： 计算未通过的学生数
n_failed = n_students - n_passed

# TODO： 计算通过率
grad_rate = 100*n_passed / (1.0 * n_students)

# 输出结果
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

(395, 31)
Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## 数据准备
在这个部分中，我们将要为建模、训练和测试准备数据
### 识别特征和目标列
你获取的数据中通常都会包含一些非数字的特征，这会导致一些问题，因为大多数的机器学习算法都会期望输入数字特征进行计算。

运行下面的代码单元将学生数据分成特征和目标列看一看他们中是否有非数字特征。

In [3]:
# 提取特征列
feature_cols = list(student_data.columns[:-1])

# 提取目标列 ‘passed’
target_col = student_data.columns[-1] 

# 显示列的列表
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# 将数据分割成特征数据和目标数据（即X_all 和 y_all）
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# 通过打印前5行显示特征信息
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### 预处理特征列

正如你所见，我们这里有几个非数值的列需要做一定的转换！它们中很多是简单的`yes`/`no`，比如`internet`。这些可以合理地转化为`1`/`0`（二元值，binary）值。

其他的列，如`Mjob`和`Fjob`，有两个以上的值，被称为_分类变量（categorical variables）_。处理这样的列的推荐方法是创建和可能值一样多的列（如：`Fjob_teacher`，`Fjob_other`，`Fjob_services`等），然后将其中一个的值设为`1`另外的设为`0`。

这些创建的列有时候叫做 _虚拟变量（dummy variables）_，我们将用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)函数来完成这个转换。运行下面代码单元的代码来完成这里讨论的预处理步骤。

In [4]:
def preprocess_features(X):
    ''' 预处理学生数据，将非数字的二元特征转化成二元值（0或1），将分类的变量转换成虚拟变量
    '''
    
    # 初始化一个用于输出的DataFrame
    output = pd.DataFrame(index = X.index)

    # 查看数据的每一个特征列
    for col, col_data in X.iteritems():
        
        # 如果数据是非数字类型，将所有的yes/no替换成1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # 如果数据类型是类别的（categorical），将它转换成虚拟变量
        if col_data.dtype == object:
            # 例子: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # 收集转换后的列
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
# print X_all

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### 实现: 将数据分成训练集和测试集
现在我们已经将所有的 _分类的（categorical）_ 特征转换成数值了。下一步我们将把数据（包括特征和对应的标签数据）分割成训练集和测试集。在下面的代码单元中，你需要完成下列功能：
- 随机混洗（shuffle）切分数据(`X_all`, `y_all`) 为训练子集和测试子集。
  - 使用300个数据点作为训练集（约75%），使用95个数据点作为测试集（约25%）。
  - 如果可能的话，为你使用的函数设置一个`random_state`。
  - 将结果存储在`X_train`, `X_test`, `y_train`和 `y_test`中。

In [15]:
# TODO：在这里导入你可能需要使用的另外的功能
from sklearn.cross_validation import StratifiedShuffleSplit
# TODO：设置训练集的数量
num_train = 300

# TODO：设置测试集的数量
num_test = X_all.shape[0] - num_train

# TODO：把数据集混洗和分割成上面定义的训练集和测试集
# X_train ,X_test ,y_train ,y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=95)
sp = StratifiedShuffleSplit(y_all, test_size=num_test, random_state=0)
for train_index, test_index in sp:
    X_train, X_test = X_all.iloc[train_index], X_all.iloc[test_index]
    y_train ,y_test = y_all[train_index], y_all[test_index]


# 显示分割的结果
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])


# print X_train.head()

Training set has 300 samples.
Testing set has 95 samples.


## 训练和评价模型
在这个部分，你将选择3个适合这个问题并且在`scikit-learn`中已有的监督学习的模型。首先你需要说明你选择这三个模型的原因，包括这些数据集有哪些特点，每个模型的优点和缺点各是什么。然后，你需要将这些模型用不同大小的训练集（100个数据点，200个数据点，300个数据点）进行训练，并用F<sub>1</sub>的值来衡量。你需要制作三个表，每个表要显示训练集大小，训练时间，预测时间，训练集上的F<sub>1</sub>值和测试集上的F<sub>1</sub>值（每个模型一个表）。

**这是目前** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **里有的监督学习模型，你可以从中选择:**
- Gaussian Naive Bayes (GaussianNB) 朴素贝叶斯
- Decision Trees 决策树
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM) 向量模型机
- Logistic Regression 逻辑回归

### 问题 2 - 应用模型
*列出三个适合这个问题的监督学习算法模型。每一个你选择的模型：*

- 描述一个该模型在真实世界的一个应用场景。（你需要为此做点研究，并给出你的引用出处）
- 这个模型的优势是什么？他什么情况下表现最好？
- 这个模型的缺点是什么？什么条件下它表现很差？
- 根据我们当前数据集的特点，为什么这个模型适合这个问题。

**回答: **
1.SVM：
SVM可以应用于图像分类任务中，“比起传统的查询优化方案，支持向量机能够取得明显更高的搜索准确度”--（https://zh.wikipedia.org/wiki/%E6%94%AF%E6%8C%81%E5%90%91%E9%87%8F%E6%9C%BA#.E5.BA.94.E7.94.A8）

SVM可以解决高维问题，并且根据VC维理论，SVM不容易过拟合，因此具有很好的泛华性能；通常SVM在小样本情况下会有很好的表现（“小”以覆盖SV点为限度）

SVM的缺点在于其对于缺失数据具有较高的敏感性，且不存在一种通用的方法解决非线性问题，实现非线性的SVM分类器时需要谨慎选择核函数。

当前数据集具有30个特征向量，属于特征维度较高的数据集，因此考虑应用SVM模型解决

2.决策树
医学诊断是决策树的一种应用--http://www.ibm.com/developerworks/cn/analytics/library/ba-1507-decisiontree-algorithm/index.html

决策树的熵理论容易理解和解释，因此决策树模型的学习过程相对直观；对于大型的数据源，决策树可以在相对短的时间内完成效果很好的训练

决策树的缺点在于训练过程容易出现过拟合，并且模型无法表达各特征之间的关联;当没有在合适的时机停止训练或对训练后的决策树进行剪枝操作时，决策树非常容易出现过拟合问题，此时决策树对于测试集的表现通常会很差

由于当前数据集的特征已经被预处理成二值化特征，因此考虑该数据集的情况很适合用来构造决策树模型；

3.AdaBoost
AdaBoost属于集成方法，可以是不同算法的集成或是同一算法在不同设置下的集成，因此AdaBoost几乎能应用于任何其集成的算法的应用领域。--《机器学习实战》第七章（前半句是原文，后半句是我的归纳理解）  例如应用于人脸识别等图像识别场景--http://blog.csdn.net/watkinsong/article/details/7631241 

由于Adaboost算法提供的是框架，因此可以使用各种方法构建子分类器，分类精度高，泛化效果好都是AdaBoost的优势；由于其可以组合各类算法的天然优势，大部分情况下Adaboost都有很好的表现

AdaBoost对离群点敏感，这种情况下其表现略差

AdaBoost通常用于二分类或多分类场景，因此考虑使用该模型处理本问题


### 准备
运行下面的代码单元以初始化三个帮助函数，这三个函数将能够帮你训练和测试你上面所选择的三个监督学习算法。这些函数是：
- `train_classifier` - 输入一个分类器和训练集，用数据来训练这个分类器。
- `predict_labels` - 输入一个训练好的分类器、特征以及一个目标标签，这个函数将帮你做预测并给出F<sub>1</sub>的值.
- `train_predict` - 输入一个分类器以及训练集和测试集，它可以运行`train_clasifier`和`predict_labels`.
 - 这个函数将分别输出训练集的F<sub>1</sub>值和测试集的F<sub>1</sub>值

In [16]:
def train_classifier(clf, X_train, y_train):
    ''' 用训练集训练分类器 '''
    
    # 开始计时，训练分类器，然后停止计时
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' 用训练好的分类器做预测并输出F1值'''
    
    # 开始计时，作出预测，然后停止计时
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # 输出并返回结果
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' 用一个分类器训练和预测，并输出F1值 '''
    
    # 输出分类器名称和训练集大小
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # 训练一个分类器
    train_classifier(clf, X_train, y_train)
    
    # 输出训练和测试的预测结果
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### 练习: 模型评价指标
借助于上面定义的函数，你现在需要导入三个你选择的监督学习模型，然后为每一个模型运行`train_predict`函数。请记住，对于每一个模型你需要在不同大小的训练集（100，200和300）上进行训练和测试。所以，你在下面应该会有9个不同的输出（每个模型都有训练集大小不同的三个输出）。在接下来的代码单元中，你将需要实现以下功能：
- 引入三个你在上面讨论过的监督式学习算法模型。
- 初始化三个模型并将它们存储在`clf_A`， `clf_B` 和 `clf_C`中。
 - 如果可能对每一个模型都设置一个`random_state`。
 - **注意:** 这里先使用每一个模型的默认参数，在接下来的部分中你将需要对某一个模型的参数进行调整。
- 创建不同大小的训练集用来训练每一个模型。
 - *不要再混洗和再分割数据！新的训练集要取自`X_train`和`y_train`.*
- 对于每一个模型要用不同大小的训练集来训练它，然后在测试集上做测试（总共需要9次训练测试）   
**注意:** 在下面的代码单元后面我们提供了三个表用来存储你的结果。

In [20]:
# TODO：从sklearn中引入三个监督学习模型
# from sklearn import model_A
from sklearn import svm
# from sklearn import model_B
from sklearn import tree
# from skearln import model_C
from sklearn.ensemble import AdaBoostClassifier

# TODO：初始化三个模型
clf_A = svm.SVC(random_state=95)
clf_B = tree.DecisionTreeClassifier(random_state=95)
clf_C = AdaBoostClassifier(random_state=95)

# TODO：设置训练集大小
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

X_train ,X_test ,y_train ,y_test
# TODO：对每一个分类器和每一个训练集大小运行'train_predict' 
for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
        print '\n'

Training a SVC using a training set size of 100. . .
Trained model in 0.0055 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8383.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.8050.


Training a SVC using a training set size of 200. . .
Trained model in 0.0086 seconds
Made predictions in 0.0027 seconds.
F1 score for training set: 0.8371.
Made predictions in 0.0014 seconds.
F1 score for test set: 0.8344.


Training a SVC using a training set size of 300. . .
Trained model in 0.0271 seconds
Made predictions in 0.0170 seconds.
F1 score for training set: 0.8677.
Made predictions in 0.0018 seconds.
F1 score for test set: 0.8082.


Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0021 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7176.


Training a DecisionTreeClassifier using a training set size of 200. . .
Trained m

### 结果表格
编辑下面的表格看看在[Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables)中如何设计一个表格。你需要把上面的结果记录在表格中。

** 分类器 1 - SVC**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0055           |        0.0008          |      0.8383      |    0.8050       |
| 200               |        0.0086           |        0.0014          |      0.8371      |    0.8344       |
| 300               |        0.0271           |        0.0018          |      0.8677      |    0.8082       |

** 分类器 2 - DecisionTreeClassifier**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0021           |        0.0002          |      1.0000      |    0.7176       |
| 200               |        0.0014           |        0.0002          |      1.0000      |    0.7176       |
| 300               |        0.0019           |        0.0002          |      1.0000      |    0.7040       |

** 分类器 3 - AdaBoostClassifier**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0620           |        0.0044          |      0.9722      |    0.7333       |
| 200               |        0.0898           |        0.0042          |      0.8746      |    0.7820       |
| 300               |        0.0717           |        0.0042          |      0.8505      |    0.8175       |

## 选择最佳模型
在最后这一部分中，你将从三个监督学习模型中选择一个用在学生数据上的最佳模型。然后你将在最佳模型上用全部的训练集（`X_train`和`y_train`）运行一个网格搜索算法，在这个过程中，你要至少调整一个参数以提高模型的F<sub>1</sub>值（相比于没有调参的模型的分值有所提高）。 

### 问题 3 - 选择最佳模型
*给予你上面做的实验，用一到两段话，向（学校）监事会解释你将选择哪个模型作为最佳的模型。哪个模型在现有的数据，有限的资源、开支和模型表现综合来看是最好的选择？*

**回答: **
综合分析如上表格，我选择AdaBoost作为模型选择的最佳选项，原因如下：

1.从算法训练时间来看：虽然前两个分类器的训练时间远远优于第三个分类器，但可以看到，SVM的训练时间，与样本数量呈线性关系，而随着训练集的增多，AdaBoost的训练时间几乎没有很大变化，考虑到实际应用情况下大量的训练数据，我认为AdaBoost是较好的选择

2.从算法预测时间来看：决策树的效果最好，SVM次之，AdaBoost最差，但和SVM的预测时间基本在同一个数量级上

3.从算法的F1评分来看：在没有特别调整决策树参数的情况下，可以看到决策树是非常容易出现过拟合问题的，训练分数很高但测试分数较低；SVM的表现相对很好很稳定，且在100个训练集的条件下也获得了很不错的训练和测试评分，这也印证了问题2中SVM的优点之一--小样本情况下的出色表现，但也应注意到，当样本数上升至300时，SVM出现了过拟合的趋势--训练评分上升而测试评分回落；在训练样本相对较少的情况下，AdaBoost的F1评分不尽如意，出现了过拟合问题，但随着训练样本的增多，这个问题得到了很好的改善，可以看到当训练样本的数目达到300的情况下，AdaBoost甚至获得了略优于SVM的测试评分，该实验结果也印证了之前课程的理论--随着训练样本的增多，几乎不需要担心集成算法出现过拟合问题

综合考虑上述因素，我会选择AdaBoost作为最佳模型

### 问题 4 - 用通俗的语言解释模型
*用一到两段话，向（学校）监事会用外行也听得懂的话来解释最终模型是如何工作的。你需要解释所选模型的主要特点。例如，这个模型是怎样被训练的，它又是如何做出预测的。避免使用高级的数学或技术术语，不要使用公式或特定的算法名词。*

**回答: **
日常生活中，当我们要在一个未知领域做出一个决定的时候，多数情况下我们会综合吸取多个专家而非某一个专家的意见，并归纳得出自己的判断，AdaBoost算法和上述流程异曲同工。

AdaBoost模型是这样反复训练的：
对于所有的学生数据，先让专家1针对学生的各项特点给出各位学生是否可以毕业的预判，如果专家1的预测结果有一半以上是正确的，我们就采纳这位专家的意见，并邀请专家2来预判专家1预测错误的部分，同样的，如果专家2的预测正确率也超过50%，那么我们采纳这位专家的意见，并反复上述过程

AdaBoost模型是这样做出预测的：
模型完成了上述的训练过程后，我们也就邀请到了专家1,2,3...，最终我们会综合各位专家的意见，通过对某一学生的特点综合分析，给出该学生是否可能毕业的预判。

### 练习: 模型调参
细调选择的模型的参数。使用网格搜索（`GridSearchCV`）来至少调整模型的重要参数（至少调整一个），这个参数至少需给出并尝试3个不同的值。你要使用整个训练集来完成这个过程。在接下来的代码单元中，你需要实现以下功能：
- 导入 [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) 和 [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- 创建一个对于这个模型你希望调整参数的字典。
 - 例如: `parameters = {'parameter' : [list of values]}`。
- 初始化你选择的分类器，并将其存储在`clf`中。
- 使用`make_scorer` 创建F<sub>1</sub>评分函数并将其存储在`f1_scorer`中。
 - 需正确设定参数`pos_label`的值！
- 在分类器`clf`上用`f1_scorer` 作为评价函数运行网格搜索,并将结果存储在`grid_obj`中。
- 用训练集(`X_train`, `y_train`)训练grid search object,并将结果存储在`grid_obj`中。

In [24]:
# TODO: 导入 'GridSearchCV' 和 'make_scorer'

from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
# TODO：创建你希望调整的参数列表
parameters = {"n_estimators" : [5, 10, 15, 20],"learning_rate" :   [0.01, 0.50, 0.60, 0.70]}

# TODO：初始化分类器
clf = AdaBoostClassifier(random_state=95)

# TODO：用'make_scorer'创建一个f1评分函数
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO：在分类器上使用f1_scorer作为评分函数运行网格搜索
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring =f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
# TODO：用训练集训练grid search object来寻找最佳参数
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
# 得到预测的结果
clf = grid_obj.best_estimator_

print clf.get_params(), '\n'

# Report the final F1 score for training and testing after parameter tuning
# 输出经过调参之后的训练集和测试集的F1值
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

{'n_estimators': 5, 'base_estimator': None, 'random_state': 95, 'learning_rate': 0.5, 'algorithm': 'SAMME.R'} 

Made predictions in 0.0010 seconds.
Tuned model has a training F1 score of 0.8223.
Made predictions in 0.0006 seconds.
Tuned model has a testing F1 score of 0.8552.


### 问题 5 - 最终的 F<sub>1</sub> 值
*最终模型的训练和测试的F<sub>1</sub>值是多少？这个值相比于没有调整过参数的模型怎么样？*

**回答: **
根据sklearn中AdaBoost的API描述，参数n_estimators和learning_rate是一对需要做出权衡的参数，非常适合用网格搜索法找出给定取值范围的最优搭配。

经过参数优化后，模型的训练和测试的F1评分分别是0.8223和0.8552，调整之前对应的评分分别是0.8505和0.8175。相比较下，调整过参数的模型的训练评分虽然有所下降，但测试评分却提升了3.77%，这意味着在n_estimators为5，learning_rate为0.5的情况下，模型的泛化能力得到了提高。

> **注意**: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。  